In [1]:
import tflearn
import tensorflow as tf

import pandas as pd
from sklearn import preprocessing
import chess_game as cg
import numpy as np



/Users/pschale/anaconda/envs/tf/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
#this is the same setup for the NN as in the other notebook
#current method of loading save only loads the values of the variables; everything has to be set up before that

board = tflearn.layers.core.input_data(shape=(None, 64, 8), name='board_input')
aux = tflearn.layers.core.input_data(shape=(None, 7), name='aux_input')
Y = tflearn.layers.core.input_data(shape=(None, 3), name='labels')

board_input_reshaped = tf.reshape(board, [-1, 8, 8, 8])

conv4 = tflearn.conv_2d(board_input_reshaped, 32, 4, activation='relu')


conv2 = tflearn.conv_2d(board_input_reshaped, 32, 2, activation='relu')
conv2_4 = tflearn.conv_2d(conv2, 64, 4, activation='relu')
# note: this is how to get separate branches of a larger NN and then have them converge
fc1 = tflearn.layers.core.fully_connected(tf.concat([conv4, conv2_4], 3), n_units=1000, activation='relu')

fc2 = tflearn.layers.core.fully_connected(tf.concat([fc1, aux], 1), n_units=2000, activation='relu')

net = tflearn.layers.core.fully_connected(fc2, n_units=3, activation='softmax')

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss)

init = tf.global_variables_initializer()

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [3]:
# starting session and loading variables
saver = tf.train.Saver()

sess = tf.Session()
#saver.restore(sess, "/Users/pschale/pythonstuff/chess_ai_project/test_saved_CNN")
saver.restore(sess, "./test_saved_CNN")

INFO:tensorflow:Restoring parameters from ./test_saved_CNN


In [5]:
# plays a game and saves the game to a numpy file
# the format it's saved in probably isn't ideal for training
# to avoid getting stuck in dumb loops, this randomly picks from the top 5 moves

a = cg.game_board()
board_positions_ar = np.empty((0, 64, 8))
aux_ar = np.empty((0,7))
score = None
for i in range(100): #do first 10 moves for each side
    next_boards = a.find_all_next_board_positions()
    next_inputs = [ele.get_NN_inputs() for ele in next_boards]
    batch_board = np.array([ele[0] for ele in next_inputs])
    batch_aux = np.array([ele[1] for ele in next_inputs])
    evaluation = sess.run(net, feed_dict={board: batch_board, aux: batch_aux})
    #chosen_movenum = np.argmax(evaluation, 0)[-2*(int(a.white_tomove) - 1)]
    chosen_movenum = np.random.choice(np.argsort(evaluation[:, -2*(int(a.white_tomove) - 1)])[:5])
    a = next_boards[chosen_movenum]
    
    nextboard_inputs = a.get_NN_inputs()
    board_positions_ar = np.append(board_positions_ar, [nextboard_inputs[0]], axis=0)
    aux_ar = np.append(aux_ar, [nextboard_inputs[1]], axis=0)
    game_end = a.game_over()
    if game_end[0]:
        score = game_end[0]
        break
        
if score:
    score = int((-2)*score + 2)
else:
    material = a.to_csv_format()[-2:]
    if material[0] > material[1]:
        score = 0
    elif material[1] > material[0]:
        score = 2
    else:
        score = 1
        
score_onehot = np.zeros((board_positions_ar.shape[0],3))
score_onehot[:,score] = 1

game_array = np.array([board_positions_ar, aux_ar, score])

print(a)

np.save('ai_games/ai_game_1', game_array)

|   | n |   |   | k |   |   | K
|   |   |   | p | B |   |   |  
|   |   |   | P |   |   |   |  
| p |   | p |   | P |   |   |  
|   |   | P |   |   |   |   |  
| P |   |   |   |   | r | P |  
| b |   |   |   |   |   |   |  
|   |   |   |   |   |   |   |  


In [ ]:
print(a)

In [ ]:
score